# Simulations and moments

This script computes the simulations and associated moments of several version of the model, with and without self-fulfilling shocks, with short and long debt, and with the floating rate bond.

In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, ".."))

using LTBonds
using Random 
using PrettyTables

  Activating environment at `~/Github/floating-rate-debt/Project.toml`


In [2]:
SAVE_MOMENTS = false # set to true to save the moments to file. 

false

In [3]:
benchmark_parameters =  let
    R = 1.01
    β = 0.9540232420
    pref = Preferences(β = β, u = make_CRRA(ra = 2))
    y = discretize(YProcess(n = 50, ρ = 0.948503, std = 0.027092, μ = 0.0, span = 3.0, tails = false))
    m = MTruncatedNormal(; std = 0.01, span = 2.0, quadN = 100)
    penalty = DefCosts(pen1 = -0.1881927550, pen2 = 0.2455843389, quadratic = true, reentry = 0.0385)
    η=0.1

    (R=R, pref=pref,  y=y, m=m, penalty=penalty, η=η )
end

(R = 1.01, pref = Preferences{CRRA2, Float64}(0.954023242, CRRA2), y = YDiscretized{YProcess{Float64, Int64}, Vector{Float64}, Matrix{Float64}, Float64}(YProcess{Float64, Int64}(50, 0.948503, 0.027092, 0.0, 3.0, false), [0.7736940032094284, 0.7818391711984078, 0.7900700885421625, 0.7983876579785925, 0.8067927917492923, 0.8152864116996035, 0.8238694493797185, 0.8325428461468501, 0.8413075532684764, 0.8501645320266717  …  1.176242906318547, 1.1886259621882678, 1.2011393823491128, 1.2137845392287432, 1.2265628197032372, 1.239475625249198, 1.2525243720974624, 1.2657104913884287, 1.279035429329019, 1.2925006473512937], [0.18120181525570228 0.12535504447193643 … 7.742222869464416e-72 1.1380889180175488e-74; 0.20275862347026066 0.16134314683122294 … 7.409028633939995e-69 1.2544237399520442e-71; … ; 0.0 0.0 … 0.1613431468312228 0.20275862347026077; 0.0 0.0 … 0.12535504447193643 0.18120181525570228], 1.0036640682963884), m = MTruncatedNormal{Float64, Distributions.Truncated{Distributions.Normal

In [4]:
mFR, mFRlowκ, mEGLT, mCKLT, mEGST, mCKST = let
    R, pref, y, m, penalty, η = benchmark_parameters

    bondFR = FloatingRateBond(;n = 350, min = 0.0, max = 1.5, λ = 0.05, κbar = 1.0) 
    bondFRlowκ = FloatingRateBond(;n = 350, min = 0.0, max = 1.5, λ = 0.05, κbar = 0.015) 
    bondLT = Bond(n = 350, min = 0.0, max = 1.5, κ = R - 1, λ = 0.05)  
    bondST = Bond(n = 350, min = 0.0, max = 1.5, κ = R - 1, λ = 1.0)  

    fr = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondFR, 
        def_costs = penalty, 
        R = R,
        η = η
    )   

    frlowκ = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondFRlowκ, 
        def_costs = penalty, 
        R = R,
        η = η
    )   

    egLT = LTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondLT, 
        def_costs = penalty, 
        R = R,
    )

    ckLT = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondLT, 
        def_costs = penalty, 
        R = R,
        η = η
    )

    egST = LTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondST, 
        def_costs = penalty, 
        R = R,
    )

    ckST = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondST, 
        def_costs = penalty, 
        R = R,
        η = η
    )

    (generate_workspace(fr), generate_workspace(frlowκ), generate_workspace(egLT), generate_workspace(ckLT), generate_workspace(egST), generate_workspace(ckST) )
end;

In [5]:
for m ∈ (mFR, mFRlowκ, mEGST, mCKST, mEGLT, mCKLT)
    @time solve!(m; print_every = 100, max_iters = 5000)
end

1: (v = 1.4304966712931169, κ = 0.99, q = 0.6091402083520817, vD = 0.5336647908767098)
101: (v = 0.5946309463050703, κ = 0.8783079426373568, q = 0.859006797596954, vD = 0.0016515421610243664)
201: (v = 0.5578743684363232, κ = 0.7464879959243917, q = 0.6999957288543697, vD = 0.00010840679994217339)
301: (v = 0.0006939164342973925, κ = 0.0005165681252385923, q = 7.086682121759669e-5, vD = 8.054937154611252e-6)
401: (v = 2.518262185446929e-7, κ = 1.3481842908724673e-7, q = 9.596437977599237e-9, vD = 9.789055255282619e-8)
501: (v = 2.0121184718391305e-9, κ = 9.525344957239668e-10, q = 5.384348522596838e-11, vD = 9.34804234020703e-10)
566: (v = 9.85771464456775e-11, κ = 4.665023922711953e-11, q = 2.495115225542577e-12, vD = 4.581224288813246e-11)
Converged.
181.449725 seconds (21.82 M allocations: 1.228 GiB, 0.17% gc time, 7.66% compilation time)
1: (v = 1.4304966712931169, κ = 0.0049999999999999906, q = 0.6091402083520817, vD = 0.5336647908767098)
101: (v = 0.006499144150183156, κ = 0.0014

In [6]:
big_T = 20_000 
big_N = 1_000
rng = Random.seed!(1234)

MersenneTwister(1234)

In [7]:
@time shocks, paths = create_shocks_paths(mCKST, big_T, big_N; rng) 

  2.796233 seconds (2.40 M allocations: 2.954 GiB, 15.50% gc time, 40.08% compilation time)


(LTBonds.ADraw{Vector{Float64}, Vector{Bool}, BitVector, Vector{Int64}}[LTBonds.ADraw{Vector{Float64}, Vector{Bool}, BitVector, Vector{Int64}}([0.008673472019512456, -0.009017438158568172, -0.004944787535042339, -0.009029142938652417, 0.008644013132535154, 0.005328132821695382, -0.0027173539603462067, 0.005023344963886674, -0.005169836206932687, -0.005605013381807765  …  -0.011573294792080335, 0.010410779780382661, 0.01282136481246118, 0.010037585781235812, -0.0126068203858304, -0.0178501990546201, 0.006447871899871372, 0.009132079754720083, -0.006678280278320479, 0.0017315200668511733], Bool[0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 1, 6, 4, 3, 9, 7, 8, 5  …  45, 43, 45, 43, 39, 36, 28, 30, 29, 30]), LTBonds.ADraw{Vector{Float64}, Vector{Bool}, BitVector, Vector{Int64}}([-0.010812327459075935, 0.0076412434691020305, -0.0011967311504935047, 0.0030924510171361553, 0.0058410328953801315, -0.

In [8]:
##EGST
@time simulation!(paths, shocks, mEGST; n = big_T, trim = 1000, trim_def = 20)
@time moments_EGST = moments(paths, mEGST)

  0.653277 seconds (535.41 k allocations: 32.254 MiB, 18.20% compilation time)
  1.804006 seconds (440.79 k allocations: 307.396 MiB, 19.53% compilation time)


(mean_bp_y = 0.8205159370054294, mean_mv_y = 0.8200171111666007, mean_spread = 0.0026233641121851054, std_spread = 0.0038417273670220088, mean_κ = 0.010000000000000005, std_κ = 3.469447045935555e-18, max_κ = 0.010000000000000009, std_c_y = 1.1327954772733293, cor_tb_y = -0.2230107318689273, cor_r_y = -0.4443599974228486, cor_r_b_y = -0.22020877354518212, cor_r_tb = 0.852039700901068, def_rate = 0.0025338422444237674, run_share = 0.0)

In [9]:
##CKST
@time simulation!(paths, shocks, mCKST; n = big_T, trim = 1000, trim_def = 20)
@time moments_CKST = moments(paths, mCKST)

  0.412039 seconds (233.72 k allocations: 13.931 MiB, 21.12% compilation time)
  1.892297 seconds (43.07 k allocations: 285.204 MiB, 16.28% gc time, 3.19% compilation time)


(mean_bp_y = 0.3751558203163908, mean_mv_y = 0.37496371481759716, mean_spread = 0.0022717464975280173, std_spread = 0.00254337713285425, mean_κ = 0.010000000000000009, std_κ = 0.0, max_κ = 0.010000000000000009, std_c_y = 1.054563714808554, cor_tb_y = -0.18687413226443453, cor_r_y = -0.6647269565487111, cor_r_b_y = -0.4930961969257421, cor_r_tb = 0.6859328736366016, def_rate = 0.0021733632391101043, run_share = 1.0)

In [10]:
##EGLT
@time simulation!(paths, shocks, mEGLT; n = big_T, trim = 1000, trim_def = 20)
@time moments_EGLT = moments(paths, mEGLT)

  0.129035 seconds (7.01 k allocations: 1000.359 KiB)
  1.016320 seconds (5 allocations: 154.038 MiB)


(mean_bp_y = 0.9360020747426997, mean_mv_y = 0.7227871856106691, mean_spread = 0.07979796161597463, std_spread = 0.044289480756940475, mean_κ = 0.010000000000000007, std_κ = 1.734723561896697e-18, max_κ = 0.010000000000000009, std_c_y = 1.1048794612942947, cor_tb_y = -0.39012460259904885, cor_r_y = -0.6476444655531318, cor_r_b_y = -0.02688692649717031, cor_r_tb = 0.7283685231028937, def_rate = 0.06675747290656353, run_share = 0.0)

In [11]:
##CKLT
@time simulation!(paths, shocks, mCKLT; n = big_T, trim = 1000, trim_def = 20)
@time moments_CKLT = moments(paths, mCKLT)

  0.123762 seconds (7.01 k allocations: 1000.391 KiB)
  1.117823 seconds (5 allocations: 154.076 MiB)


(mean_bp_y = 0.9358849194612805, mean_mv_y = 0.7227215425873066, mean_spread = 0.07978253884172508, std_spread = 0.04425432552008048, mean_κ = 0.010000000000000004, std_κ = 5.204170685625763e-18, max_κ = 0.010000000000000009, std_c_y = 1.1049216359382137, cor_tb_y = -0.3900731572747227, cor_r_y = -0.64784785581592, cor_r_b_y = -0.02756820585698036, cor_r_tb = 0.7285045267699705, def_rate = 0.06671753670178815, run_share = 0.0025253817933623413)

In [12]:
##FR
@time simulation!(paths, shocks, mFR; n = big_T, trim = 1000, trim_def = 20)
@time moments_FR = moments(paths, mFR)

  0.468065 seconds (251.10 k allocations: 14.922 MiB, 19.14% compilation time)
  1.643607 seconds (43.06 k allocations: 284.329 MiB, 3.70% compilation time)


(mean_bp_y = 0.7654470687661199, mean_mv_y = 0.765447068762887, mean_spread = 0.002543287936540909, std_spread = 0.003524822702660151, mean_κ = 0.01061548189703876, std_κ = 0.0008507161946958987, max_κ = 0.02298936895975756, std_c_y = 1.1060128057804124, cor_tb_y = -0.2160053117317691, cor_r_y = -0.40449285271178614, cor_r_b_y = -0.1380279721323514, cor_r_tb = 0.7405112964057511, def_rate = 0.0024445507067005767, run_share = 0.774761230986912)

In [13]:
##FRlowκ
@time simulation!(paths, shocks, mFRlowκ; n = big_T, trim = 1000, trim_def = 20)
@time moments_FRlowκ = moments(paths, mFRlowκ)

  0.197818 seconds (7.01 k allocations: 1000.359 KiB)
  1.261239 seconds (5 allocations: 206.611 MiB)


(mean_bp_y = 0.8711594132199812, mean_mv_y = 0.775639666300322, mean_spread = 0.0376579505791321, std_spread = 0.028733836098617835, mean_κ = 0.011108334439004865, std_κ = 0.0017227599955178642, max_κ = 0.015, std_c_y = 1.1183325594377467, cor_tb_y = -0.3038392953097517, cor_r_y = -0.4355863759097389, cor_r_b_y = 0.03466740773216423, cor_r_tb = 0.5654804987127153, def_rate = 0.03289459050226229, run_share = 0.0028496292843384727)

In [14]:
pretty_table(
    [
        pairs(moments_EGST),
        pairs(moments_CKST),
        pairs(moments_EGLT),
        pairs(moments_CKLT),
        pairs(moments_FR),
        pairs(moments_FRlowκ)
    ],
    row_names = ["EGST", "CKST", "EGLT", "CKLT", "FR", "FRlowκ"]
)

┌────────┬───────────┬───────────┬─────────────┬────────────┬───────────┬─────────────┬───────────┬─────────┬───────────┬───────────┬────────────┬──────────┬────────────┬────────────┐
│        │ mean_bp_y │ mean_mv_y │ mean_spread │ std_spread │    mean_κ │       std_κ │     max_κ │ std_c_y │  cor_tb_y │   cor_r_y │  cor_r_b_y │ cor_r_tb │   def_rate │  run_share │
├────────┼───────────┼───────────┼─────────────┼────────────┼───────────┼─────────────┼───────────┼─────────┼───────────┼───────────┼────────────┼──────────┼────────────┼────────────┤
│   EGST │  0.820516 │  0.820017 │  0.00262336 │ 0.00384173 │      0.01 │ 3.46945e-18 │      0.01 │  1.1328 │ -0.223011 │  -0.44436 │  -0.220209 │  0.85204 │ 0.00253384 │        0.0 │
│   CKST │  0.375156 │  0.374964 │  0.00227175 │ 0.00254338 │      0.01 │         0.0 │      0.01 │ 1.05456 │ -0.186874 │ -0.664727 │  -0.493096 │ 0.685933 │ 0.00217336 │        1.0 │
│   EGLT │  0.936002 │  0.722787 │    0.079798 │  0.0442895 │      0.01 │ 1.7347

In [15]:
SAVE_MOMENTS && open(joinpath(@__DIR__,"..","output","moments.txt"), "w") do f
    pretty_table(f,
    [
        pairs(moments_EGST),
        pairs(moments_CKST),
        pairs(moments_EGLT),
        pairs(moments_CKLT),
        pairs(moments_FR),
        pairs(moments_FRlowκ)
    ],
    row_names = ["EGST", "CKST", "EGLT", "CKLT", "FR", "FRlowκ"]
)
end

false